In [83]:
!pip freeze | grep scikit-learn

scikit-learn==1.5.0


In [2]:
!python -V

Python 3.12.3


In [26]:
import pickle
import pandas as pd
import numpy as np

In [6]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [14]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [69]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')

In [70]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [71]:
y_pred.std()

6.247488852238703

In [73]:
df['ride_id'] = f'2023/03_' + df.index.astype('str')
df['ride_id']

0                2023/03_0
1                2023/03_1
2                2023/03_2
3                2023/03_3
4                2023/03_4
                ...       
3403761    2023/03_3403761
3403762    2023/03_3403762
3403763    2023/03_3403763
3403764    2023/03_3403764
3403765    2023/03_3403765
Name: ride_id, Length: 3316216, dtype: object

In [74]:
y_pred

array([16.24590642, 26.1347962 , 11.88426424, ..., 11.59533603,
       13.11317847, 12.89999218])

,y_pred
0,16.245906
1,26.134796
2,11.884264
3,11.997720
4,10.234486
...,...
3403761,11.952635
3403762,20.049958
3403763,11.595336
3403764,13.113178


In [76]:
df_result = pd.concat([df['ride_id'], pd.DataFrame(y_pred, columns=["predictions"], index=df.index)], axis=1)
df_result

,ride_id,predictions
0,2023/03_0,16.245906
1,2023/03_1,26.134796
2,2023/03_2,11.884264
3,2023/03_3,11.997720
4,2023/03_4,10.234486
...,...,...
3403761,2023/03_3403761,11.952635
3403762,2023/03_3403762,20.049958
3403763,2023/03_3403763,11.595336
3403764,2023/03_3403764,13.113178


In [77]:
[df['ride_id'].shape, y_pred.shape, df_result.shape]

[(3316216,), (3316216,), (3316216, 2)]

In [81]:
df_result.to_parquet(
    "../tmp/df_result.parquet",
    engine='pyarrow',
    compression=None,
    index=False
)

In [82]:
!ls -lah ../tmp/df_result.parquet

-rw-r--r--@ 1 asfandqazi  staff    65M Jun 11 05:11 ../tmp/df_result.parquet
